In [1]:
import os

os.environ['OPENAI_API_KEY']="OPEN_API_KEY"

In [4]:
# from langchain.llms import OpenAI # this code has been deprecated since recording.
from langchain.chat_models import ChatOpenAI # this is the replacement 
from langchain.prompts import PromptTemplate
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from random import sample 

First, let's create a loader and load reviews from tv-reviews.csv into memory

In [6]:
# TODO: load reviews from tv-reviews.csv
loader = CSVLoader('tv-reviews.csv')
df = loader.load()

Then, let's initialize our LLM

In [34]:
# TODO: initialize OpenAI object with your API key
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125", 
    temperature=0
)

Now, let's setup our parser and a template  - 

**Note**  that since recording, the code to initialize the model has been updated to 

`llm = ChatOpenAI()`

In [35]:
class ReviewSentiment(BaseModel):
    positives: List[NonNegativeInt] = Field(description="index of a positive TV review, starting from 0")
    negatives: List[NonNegativeInt] = Field(description="index of a negative TV review, starting from 0")
        
parser = PydanticOutputParser(pydantic_object=ReviewSentiment)
# TODO: setup a template with partial and input variables
print(parser.get_format_instructions())

prompt = PromptTemplate(
    template="{question}\n{format_instructions}\nContext: {context}", 
    input_variables=['question', 'context'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

question = """
    Classify TV reviews provided in the context into positive and negative.
    Only use reviews provided in the context, do not make new reviews or use any existing information you know about these TVs.
    If there are no positive or negative reviews, output an empty JSON array
"""
reviews_to_classify = sample(df, 3)

context = "\n".join(review.page_content for review in reviews_to_classify)

query = prompt.format(context=context, question=question)
print(query)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"title": "Positives", "description": "index of a positive TV review, starting from 0", "type": "array", "items": {"type": "integer", "minimum": 0}}, "negatives": {"title": "Negatives", "description": "index of a negative TV review, starting from 0", "type": "array", "items": {"type": "integer", "minimum": 0}}}, "required": ["positives", "negatives"]}
```

    Classify TV reviews provided in the context into positive and negative.
    Only use reviews provided in the context, do not make new reviews or use any exis

Pick 3 sample reviews to classify - LLMs have a limited context window they can work with. In later exercises, we'll see how to deal with that differently

In [39]:
# TODO: pick 3 random reviews and save them into reviews_to_classify variable
reviews_to_classify = sample(df, 3)

# generate textual prompt from the prompt template
question = """
    Review TVs provided in the context. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 
"""
query = prompt.format(context = context, question = question)

Finally, let's send our query to LLM and use the parser we setup to parse an output into a Python object

**NOTE**: Since recording the code to feed the query to the llm has been updated to

`llm.predict(query)`

In [40]:
# TODO: query LLM, then parse output into the result variable
output = llm.predict(query)
result = parser.parse(output)
print(result)
print("Positives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.positives]))
print("Negatives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.negatives]))

positives=[2] negatives=[0, 1]
Positives:
TV Name: VisionMax Ultra
Review Title: Immersive Audio Experience
Review Rating: 10
Review Text: The VisionMax Ultra delivers an immersive audio experience. The built-in speakers produce clear and well-balanced sound. The Dolby Atmos technology adds depth and dimension to the audio, making it feel like I'm in a theater. I'm thoroughly impressed!
Negatives:
TV Name: VisionMax Ultra
Review Title: Impressive Smart Features
Review Rating: 8
Review Text: The VisionMax Ultra comes loaded with impressive smart features. The user interface is intuitive and easy to navigate. The TV seamlessly integrates with various streaming platforms, providing a hassle-free entertainment experience. It's like having a smart hub at my fingertips.
TV Name: Imagix Pro
Review Title: Disappointing Sound Quality
Review Rating: 3
Review Text: While the picture quality of the Imagix Pro is excellent, I am disappointed with the sound quality. The volume lacks depth and the ba